In [24]:
import subprocess
import re
import json

def get_monitor_info():
    monitors_raw = subprocess.check_output(["xrandr", "--listmonitors"], text=True).strip().split('\n')[1:]
    monitors = []
    for line in monitors_raw:
        parts = re.split(r'\s+', line.strip())
        monitor_id = parts[1].lstrip('+*')
        width_px = int(parts[2].split('+')[0].split('x')[0].split('/')[0])
        height_px = int(parts[2].split('+')[0].split('x')[1].split('/')[0])
        aspect_ratio = round(width_px / height_px, 2) if height_px else None
        monitors.append({
            "monitor_id": monitor_id,
            "resolution": f"{width_px}x{height_px}",
            "aspect_ratio": aspect_ratio,
        })
    return monitors

def get_window_info():
    windows_raw = subprocess.check_output(["wmctrl", "-lGx"], text=True).strip().split('\n')
    windows = []
    for line in windows_raw:
        parts = line.split(None, 9)
        if len(parts) < 10:
            continue
        window_id = parts[0]
        x, y, w, h = map(int, parts[2:6])
        wm_class = parts[6]  # e.g. 'google-chrome.Google-chrome'
        
        window_name = parts[9].strip()
        aspect_ratio = round(w / h, 2) if h else None
        app_name = wm_class.split('.')[0] if wm_class else None
        windows.append({
            "window_id": window_id,
            "window_name": window_name,
            "application": app_name,
            "resolution": f"{w}x{h}",
            "aspect_ratio": aspect_ratio,
        })
    return windows

def list_monitors_and_windows_json():
    monitors = get_monitor_info()
    windows = get_window_info()
    return {"monitors": monitors, "windows": windows}


In [26]:
list_monitors_and_windows_json()

{'monitors': [{'monitor_id': 'HDMI-1',
   'resolution': '1920x1080',
   'aspect_ratio': 1.78},
  {'monitor_id': 'eDP-1', 'resolution': '1600x900', 'aspect_ratio': 1.78}],
 'windows': [{'window_id': '0x02c0001d',
   'window_name': 'Stream - Google Chrome',
   'application': 'google-chrome',
   'resolution': '1600x857',
   'aspect_ratio': 1.87},
  {'window_id': '0x03e00004',
   'window_name': 'playground.ipynb - Untitled (Workspace) - Visual Studio Code',
   'application': 'code',
   'resolution': '1920x1052',
   'aspect_ratio': 1.83},
  {'window_id': '0x02c00075',
   'window_name': '- ESIGN-Software-Frontend [GitHub] - Visual Studio Code - GitHub - Google Chrome',
   'application': 'google-chrome',
   'resolution': '1920x1009',
   'aspect_ratio': 1.9},
  {'window_id': '0x02e09122',
   'window_name': 'Remote Desktop Client',
   'application': 'org',
   'resolution': '1920x1052',
   'aspect_ratio': 1.83},
  {'window_id': '0x02e1c4f8',
   'window_name': 'Connect',
   'application': 'org',


In [30]:
import mss

def screenshot_by_monitor_name(name="eDP-1", output="screenshot.png"):
    with mss.mss() as sct:
        for monitor in sct.monitors[1:]:  # skip all-in-one
            if name in monitor.get('name', ''):
                sct_img = sct.grab(monitor)
                mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
                return output
    raise ValueError(f"Monitor {name} not found")
screenshot_by_monitor_name()

ValueError: Monitor eDP-1 not found

In [32]:
import subprocess
import re

def get_monitor_geometry(name):
    output = subprocess.check_output(['xrandr']).decode()
    match = re.search(rf'^{name} connected.*?(\d+)x(\d+)\+(\d+)\+(\d+)', output, re.MULTILINE)
    if match:
        width, height, x, y = map(int, match.groups())
        return {"width": width, "height": height, "x": x, "y": y}
    return None

geometry = get_monitor_geometry("HDMI-1")
print(geometry)


{'width': 1920, 'height': 1080, 'x': 1600, 'y': 0}
